# Determinant analysis on experimental data from Si & Le Treut (2019)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## Imports

In [24]:
import os, copy, pickle, itertools, tabulate
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import sys
sys.path.append('code')
from functions_response import add_allvariables, compute_determinant, plot_Ivalues, load_table

## Parameters

In [26]:
direxp = os.path.join('.','experiments')
names = ['mg1655_acetate', \
         'mg1655_glucose', \
         'mg1655_glycerol11aa', \
         'ncm3722_arginine', \
         'ncm3722_glucose', \
         'ncm3722_glucose12aa' \
        ]
# names = [
#          'ncm3722_glucose'
#         ]

label_mapping = { \
                'lambda': "$\lambda$", \
                'Sb': "$S_b$",\
                'Sd': "$S_d$",\
                'tau': "$\\tau$",\
                'tau_cyc': "$\\tau_{cyc}$" , \
                'tau_bi': "$\\tau_{bi}$", \
                'Si': "$S_i$" , \
                'Delta_bd': "$\Delta_{d}$", \
                'Delta_bi': "$\Delta_{bi}$", \
                'R_bd': "$R_{bd}$", \
                'R_bi': "$R_{bi}$", \
                'Lambda_i': "$\Lambda_i$", \
                'tau_ii': "$\\tau_{ii}$", \
                'delta_id': "$\delta_{id}$", \
                'delta_ii_backward': "$\delta_{ii}^{(bwd)}$", \
                'delta_ii_forward': "$\delta_{ii}^{(fwd)}$", \
                'R_ii': "$R_{ii}$", \
                'R_id': "$R_{id}$"
                }
nval = 50  # number of values to report in the plot
ext_list = ['.png','.svg']

## Perform determinant analysis

In [5]:
for name in names:
    fpath = os.path.join(direxp,name,'colidata.pkl')
#     print(fpath)
    
    colidata = pd.read_pickle(fpath)


    sel=['lambda', 'Sb','Sd','tau','tau_cyc', 'initiator B', 'Si', 'Delta_bd', 'Delta_bi', 'R_bd', 'R_bi', 'Lambda_i', 'tau_ii', 'delta_id', 'delta_ii', 'R_ii', 'R_id', 'delta_ii_forward' ]

    add_allvariables(colidata)
#     df.loc[:,sel]

    # rename some variables
    col_mapping = { \
                'initiator B': 'tau_bi', \
                'delta_ii': 'delta_ii_backward', \
                }
    colidata = colidata.rename(columns=col_mapping)

    for i in range(len(sel)):
        if sel[i] in col_mapping.keys():
            sel[i] = col_mapping[sel[i]]
    colidata['lambda'] = colidata['lambda']*60
#     colidata.loc[:, sel]

    # determinant analysis
    count=0
    nvar = 3
    table = []
    for comb in itertools.combinations(sel, nvar):
        count+=1
        data = colidata.loc[:,list(comb)].dropna().to_numpy().astype('float64').T
        I = compute_determinant(data)
    #     print(count,comb,"I = {:.4f}".format(I))
        table += [list(comb) + [I]]

    table.sort(key=lambda x: x[3], reverse=True)
    headers=['var {:d}'.format(i+1) for i in range(nvar)]+['I']

    fname = 'determinant_analysis'
    fpath = os.path.join(direxp,name,fname + '.txt')

    with open(fpath,'w') as fout:
        fout.write(tabulate.tabulate(table, tablefmt='plain', headers=headers))
    

## Print ranking for Witz 2019, Si & Le Treut 2019 and HC models.

In [9]:
for name in names:
    print('-'*10)
    print(name)
    
    fname = 'determinant_analysis'
    fpath = os.path.join(direxp,name,fname + '.txt')
    table = load_table(fpath)

    # get the 3 values
    special_combs = {\
                     'GW (backward)': ['lambda', 'delta_ii_backward', 'delta_id'], \
                     'GW (forward)': ['lambda', 'delta_ii_forward', 'delta_id'], \
                     'HC': ['lambda', 'tau_cyc', 'Lambda_i'], \
                     'SLT (backward)': ['lambda', 'Delta_bd', 'delta_ii_backward'], \
                     'SLT (forward)': ['lambda', 'Delta_bd', 'delta_ii_forward'], \
                    }
    for key in special_combs.keys():
        myvars = set(special_combs[key])
        for line in table:
            thisvars = set(line[:-1])
            I = line[-1]
            if myvars == thisvars:
                print("{:<20s}I = {:.6f}".format(key,I))
                break

----------
mg1655_acetate
GW (backward)       I = 0.814357
GW (forward)        I = 0.924268
HC                  I = 0.417602
SLT (backward)      I = 0.648092
SLT (forward)       I = 0.829172
----------
mg1655_glucose
GW (backward)       I = 0.908310
GW (forward)        I = 0.875542
HC                  I = 0.471560
SLT (backward)      I = 0.879010
SLT (forward)       I = 0.833507
----------
mg1655_glycerol11aa
GW (backward)       I = 0.851348
GW (forward)        I = 0.812516
HC                  I = 0.497346
SLT (backward)      I = 0.769906
SLT (forward)       I = 0.713263
----------
ncm3722_arginine
GW (backward)       I = 0.800610
GW (forward)        I = 0.807433
HC                  I = 0.671608
SLT (backward)      I = 0.781593
SLT (forward)       I = 0.934928
----------
ncm3722_glucose
GW (backward)       I = 0.953348
GW (forward)        I = 0.939279
HC                  I = 0.738780
SLT (backward)      I = 0.911598
SLT (forward)       I = 0.973360
----------
ncm3722_glucose12aa
GW (ba

### Make plots

In [27]:
for name in names:
    print('-'*10)
    print(name)
    
    # load data
    fname = 'determinant_analysis'
    fpath = os.path.join(direxp,name,fname + '.txt')
    table = load_table(fpath)
    
    # make figure
    figname = "determinant_analysis_top{:d}".format(nval)
    fpath = os.path.join(direxp,name,fname)
    fig = plot_Ivalues(table, label_mapping, fig_title=name, nval=nval, figsize=(3,12), fmt_str='{:.4f}')
    
    for ext in ext_list:
        fig.savefig(fpath+ext, dpi=300, bbox_inches='tight', pad_inches=0)
        print("Saved figure in {:s}".format(fpath+ext))
#     fig.clf()
    plt.close('all')


----------
mg1655_acetate
Saved figure in ./experiments/mg1655_acetate/determinant_analysis.png
Saved figure in ./experiments/mg1655_acetate/determinant_analysis.svg
----------
mg1655_glucose
Saved figure in ./experiments/mg1655_glucose/determinant_analysis.png
Saved figure in ./experiments/mg1655_glucose/determinant_analysis.svg
----------
mg1655_glycerol11aa
Saved figure in ./experiments/mg1655_glycerol11aa/determinant_analysis.png
Saved figure in ./experiments/mg1655_glycerol11aa/determinant_analysis.svg
----------
ncm3722_arginine
Saved figure in ./experiments/ncm3722_arginine/determinant_analysis.png
Saved figure in ./experiments/ncm3722_arginine/determinant_analysis.svg
----------
ncm3722_glucose
Saved figure in ./experiments/ncm3722_glucose/determinant_analysis.png
Saved figure in ./experiments/ncm3722_glucose/determinant_analysis.svg
----------
ncm3722_glucose12aa
Saved figure in ./experiments/ncm3722_glucose12aa/determinant_analysis.png
Saved figure in ./experiments/ncm3722_gl